In [7]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('../')

from main import load_and_prepare_sessions

from collections import defaultdict
import matplotlib.pyplot as plt
import analysis.t_test

sessions = load_and_prepare_sessions("../../Baseline", load_from_pickle=True, remove_bad_signal_sessions=True)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
sorted_results = analysis.t_test.sessions_t_test(sessions)

In [9]:
import pandas as pd
# Print the sorted results

all_ps = defaultdict(list)
all_ts = defaultdict(list)

for sorted_result in sorted_results:
    if sorted_result['P-Value'] > 0.05:
        continue
    curr_key = sorted_result['key']
    p_val = sorted_result['P-Value']
    t_stat = sorted_result['T-Statistic']
    n = sorted_result['n']
    
    for metric in curr_key:
        all_ps[metric].append(p_val)
        all_ts[metric].append(t_stat)

    print(curr_key)
    print("T-Statistic:", t_stat)
    print("P-Value:", p_val)
    print("N:", n)
    print('')

from statistics import geometric_mean
# Function to calculate the mean based on the flag
def mean(values, use_geometric_mean=False, use_abs=False):
    if use_geometric_mean:
        res = geometric_mean(values)
    else:
        res = sum(values) / len(values)
    if use_abs:
        return abs(res)
    return res

# Sort all_ps by list mean (arithmetic or geometric based on the flag)
sorted_all_ps = sorted(all_ps.items(), key=lambda x: mean(x[1], use_geometric_mean=True))

# Sort all_ts by list mean (arithmetic or geometric based on the flag)
sorted_all_ts = sorted(all_ts.items(), key=lambda x: mean(x[1], use_abs=True), reverse=True)

# Converting to DataFrame for low p-value factors
df_low_p = pd.DataFrame([(factor, f'{mean(values, use_geometric_mean=True):.3e}') for factor, values in sorted_all_ps], columns=['Factor', 'Mean (Geometric)'])

# Converting to DataFrame for high t-stat factors
df_high_t = pd.DataFrame([(factor, round(mean(values), 3)) for factor, values in sorted_all_ts], columns=['Factor', 'Mean'])

# Pretty print DataFrames
print("Factors that matter the most for a low p-value:")
display(df_low_p)

print("\nFactors that matter the most for a high t-stat:")
display(df_high_t)



# Create a list of dictionaries from the sorted results
sorted_results_list = []
for sorted_result in sorted_results:
    sorted_results_list.append({
        'key': sorted_result['key'],
        'T-Statistic': sorted_result['T-Statistic'],
        'P-Value': sorted_result['P-Value'],
        'N events': sorted_result['n'] 
    })

# Create a DataFrame from the list of dictionaries
df_sorted_results = pd.DataFrame(sorted_results_list)

# Save the DataFrame to a CSV file
df_sorted_results.to_csv('sorted_p_values.csv', index=False)



('participation', 'VS', 'miss', 'maximal_value')
T-Statistic: -9.51048883989563
P-Value: 6.837253313864367e-20
N: 263

('participation', 'VS', 'cor_reject', 'maximal_value')
T-Statistic: -8.191114715317648
P-Value: 5.088039657084135e-16
N: 836

('d_prime', 'VS', 'miss', 'maximal_value')
T-Statistic: -7.44876750554364
P-Value: 3.989247300463795e-13
N: 259

('participation', 'VS', 'reward_collect', 'peak_timing')
T-Statistic: 7.6710931079106315
P-Value: 8.635971678040367e-13
N: 96

('c_score', 'VS', 'reward_collect', 'slope_down')
T-Statistic: -7.559445650536349
P-Value: 8.727494686729248e-13
N: 120

('participation', 'VS', 'cor_reject', 'peak_timing')
T-Statistic: 7.092456915140255
P-Value: 1.938315692696374e-12
N: 836

('c_score', 'VS', 'hit', 'slope_down')
T-Statistic: -6.7657578991381815
P-Value: 1.021092218523007e-10
N: 120

('d_prime', 'DLS', 'hit', 'maximal_value')
T-Statistic: -6.7416421812691425
P-Value: 1.45894335136885e-10
N: 107

('participation', 'VS', 'cor_reject', 'slope_u

,Factor,Mean (Geometric)
0,VS,8.109e-07
1,maximal_value,1.667e-06
2,cor_reject,9.435e-06
3,miss,1.383e-05
4,participation,1.978e-05
5,reward_collect,2.645e-05
6,d_prime,3.877e-05
7,hit,5.688e-05
8,c_score,5.861e-05
9,peak_timing,7.201e-05



Factors that matter the most for a high t-stat:


,Factor,Mean
0,slope_up,-1.737
1,miss,-1.262
2,peak_timing,1.225
3,participation,-1.216
4,VS,-1.173
5,maximal_value,-1.118
6,auc,-1.074
7,DLS,-0.741
8,hit,-0.579
9,slope_down,-0.571
